In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_scheduler, BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.optim as optim
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Airbnb_Open_Data.csv")
df.head()

C:\Users\ashis\AppData\Local\Temp\ipykernel_20872\1754490130.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Airbnb_Open_Data.csv")


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [3]:
# Check which column is index 25
print("Column names:")
print(df.columns.tolist())
print(f"\nColumn at index 25: {df.columns[25]}")

Column names:
['id', 'NAME', 'host id', 'host_identity_verified', 'host name', 'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country', 'country code', 'instant_bookable', 'cancellation_policy', 'room type', 'Construction year', 'price', 'service fee', 'minimum nights', 'number of reviews', 'last review', 'reviews per month', 'review rate number', 'calculated host listings count', 'availability 365', 'house_rules', 'license']

Column at index 25: license


In [4]:
# Check the data types
print(f"\nData types:\n{df.dtypes}")

# Look at the problematic column values
problematic_col = df.columns[25]
print(f"\nSample values from {problematic_col}:")
print(df[problematic_col].head(10))
print(f"\nUnique values in {problematic_col}:")
print(df[problematic_col].unique()[:20])


Data types:
id                                  int64
NAME                               object
host id                             int64
host_identity_verified             object
host name                          object
neighbourhood group                object
neighbourhood                      object
lat                               float64
long                              float64
country                            object
country code                       object
instant_bookable                   object
cancellation_policy                object
room type                          object
Construction year                 float64
price                              object
service fee                        object
minimum nights                    float64
number of reviews                 float64
last review                        object
reviews per month                 float64
review rate number                float64
calculated host listings count    float64
availability 365     

In [5]:
df.shape

(102599, 26)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [7]:
df['country code'].unique()

array(['US', nan], dtype=object)

In [8]:
df['country'].unique()

array(['United States', nan], dtype=object)

In [9]:
print(df['country code'].isnull().sum())

131


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [11]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [12]:
df.shape

(102599, 26)

In [13]:
df['last review'] = pd.to_datetime(df['last review'])
df['days_since_last_review'] = (df['last review'].max() - df['last review']).dt.days
df['days_since_last_review'] = df['days_since_last_review'].fillna(df['days_since_last_review'].max())

In [14]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license,days_since_last_review
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN,13389.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,30.0,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN,13175.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,3.0,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN,16776.0
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,30.0,270.0,2019-07-05,4.64,4.0,1.0,322.0,NaN,NaN,14226.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,10.0,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN,14454.0


In [15]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified               289
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15893
reviews per month                  15879
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              102599 non-null  int64         
 1   NAME                            102349 non-null  object        
 2   host id                         102599 non-null  int64         
 3   host_identity_verified          102310 non-null  object        
 4   host name                       102193 non-null  object        
 5   neighbourhood group             102570 non-null  object        
 6   neighbourhood                   102583 non-null  object        
 7   lat                             102591 non-null  float64       
 8   long                            102591 non-null  float64       
 9   country                         102067 non-null  object        
 10  country code                    102468 non-null  object 

In [17]:
df['host_identity_verified'].unique()

array(['unconfirmed', 'verified', nan], dtype=object)

In [18]:
df['host_identity_verified'] = df['host_identity_verified'].fillna('unconfirmed')

In [19]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15893
reviews per month                  15879
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [20]:
df['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'brookln', 'manhatan', 'Queens', nan,
       'Staten Island', 'Bronx'], dtype=object)

In [21]:
df['reviews per month'] = df['reviews per month'].fillna(0)

In [22]:
df = df.dropna(subset=['lat', 'long'])

In [23]:
df['house_rules'] = df['house_rules'].fillna("No rules provided.")

In [24]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15892
reviews per month                      0
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [25]:
df['instant_bookable'].unique()

array([False, True, nan], dtype=object)

In [26]:
df['instant_bookable'].isna().sum()/len(df)

np.float64(0.0010234815919525104)

In [27]:
df['instant_bookable'] = df['instant_bookable'].fillna(df['instant_bookable'].mode()[0])

C:\Users\ashis\AppData\Local\Temp\ipykernel_20872\296037632.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['instant_bookable'] = df['instant_bookable'].fillna(df['instant_bookable'].mode()[0])


In [28]:
df['cancellation_policy'].unique()

array(['strict', 'moderate', 'flexible', nan], dtype=object)

In [29]:
df.loc[:,'cancellation_policy'] = df['cancellation_policy'].fillna(df['cancellation_policy'].mode()[0])

In [30]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15892
reviews per month                      0
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [31]:
# Step 1: Build neighbourhood → neighbourhood_group mapping
mapping = (
    df[df['neighbourhood group'].notna()]               # take only rows with filled group
      .groupby('neighbourhood')['neighbourhood group']  # group by neighbourhood
      .agg(lambda x: x.mode()[0])                       # pick the most common group
)

# Step 2: Fill missing values using mapping
df.loc[:,'neighbourhood group'] = df['neighbourhood group'].fillna(
    df['neighbourhood'].map(mapping)
)
    

In [32]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                         16
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15892
reviews per month                      0
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [33]:
import numpy as np
from sklearn.neighbors import KDTree

# Step 1: Split data into known and unknown neighbourhood
df_known = df[df['neighbourhood'].notna()].copy()
df_unknown = df[df['neighbourhood'].isna()].copy()

# Create KDTree using coordinates of known neighbourhoods
coords_known = df_known[['lat', 'long']].values
tree = KDTree(coords_known)

# Step 2: Query nearest neighbour for rows with missing neighbourhood
coords_unknown = df_unknown[['lat', 'long']].values
dist, idx = tree.query(coords_unknown, k=1)

# Step 3: Fill neighbourhood values from nearest known point
df_unknown['neighbourhood'] = df_known.iloc[idx.flatten()]['neighbourhood'].values

# Step 4: Merge back
df = pd.concat([df_known, df_unknown]).sort_index()


In [34]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15892
reviews per month                      0
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [35]:
df['review rate number'].unique()

array([ 4.,  5.,  3., nan,  2.,  1.])

In [36]:
df.loc[df['last review'].isna(), 'review rate number'] = 0
df.loc[df['last review'].isna(), 'number of reviews'] = 0

In [37]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                     59
last review                        15892
reviews per month                      0
review rate number                   288
calculated host listings count       319
availability 365                     448
house_rules     

In [38]:
from scipy.stats import skew

def safe_skew(x):
    x = x.dropna()
    if len(x) < 2:
        return None   # or np.nan
    return skew(x, bias=False)

skew_df = (
    df.groupby(['host id', 'neighbourhood'])['review rate number']
      .apply(safe_skew)
      .reset_index(name='skewness')
)

print(skew_df)


C:\Users\ashis\AppData\Local\Temp\ipykernel_20872\175740830.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return skew(x, bias=False)


            host id     neighbourhood  skewness
0         123600518    Hell's Kitchen       NaN
1         124039648   Upper West Side       NaN
2         124472619      Williamsburg       NaN
3         129756565           Astoria       NaN
4         130349612           Chelsea       NaN
...             ...               ...       ...
102045  98758136270   Upper West Side       NaN
102046  98760968631      East Village       NaN
102047  98762661810          Flushing       NaN
102048  98762683230  Prospect Heights       NaN
102049  98763129024        Greenpoint       NaN

[102050 rows x 3 columns]


In [39]:
df.shape

(102591, 27)

In [40]:
len(df['host id'].unique())

102049

In [41]:
len(df['neighbourhood'].unique())

224

In [42]:
df.dtypes

id                                         int64
NAME                                      object
host id                                    int64
host_identity_verified                    object
host name                                 object
neighbourhood group                       object
neighbourhood                             object
lat                                      float64
long                                     float64
country                                   object
country code                              object
instant_bookable                            bool
cancellation_policy                       object
room type                                 object
Construction year                        float64
price                                     object
service fee                               object
minimum nights                           float64
number of reviews                        float64
last review                       datetime64[ns]
reviews per month   

In [43]:
df[['service fee', 'price']]

,service fee,price
0,$193,$966
1,$28,$142
2,$124,$620
3,$74,$368
4,$41,$204
...,...,...
102594,$169,$844
102595,$167,$837
102596,$198,$988
102597,$109,$546


In [44]:
for col in ['price', 'service fee']:
    df[col] = df[col].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

In [45]:
df[['service fee', 'price']]

,service fee,price
0,193.0,966.0
1,28.0,142.0
2,124.0,620.0
3,74.0,368.0
4,41.0,204.0
...,...,...
102594,169.0,844.0
102595,167.0,837.0
102596,198.0,988.0
102597,109.0,546.0


In [46]:
df.dtypes

id                                         int64
NAME                                      object
host id                                    int64
host_identity_verified                    object
host name                                 object
neighbourhood group                       object
neighbourhood                             object
lat                                      float64
long                                     float64
country                                   object
country code                              object
instant_bookable                            bool
cancellation_policy                       object
room type                                 object
Construction year                        float64
price                                    float64
service fee                              float64
minimum nights                           float64
number of reviews                        float64
last review                       datetime64[ns]
reviews per month   

In [47]:
#Pearson Correlation with review_rate_number

# Select only numeric columns (excluding id-like columns if needed)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Drop ID columns if present
drop_cols = ['id', 'host id']
numeric_cols = [c for c in numeric_cols if c not in drop_cols]

# Pearson correlation
pearson_corr = df[numeric_cols].corr()['review rate number'].sort_values(ascending=False)

print("==== Pearson Correlation with review rate number ====\n")
print(pearson_corr)


==== Pearson Correlation with review rate number ====

review rate number                1.000000
reviews per month                 0.235320
number of reviews                 0.154838
long                              0.046858
availability 365                  0.003737
Construction year                 0.003343
price                            -0.000489
service fee                      -0.000587
lat                              -0.022944
calculated host listings count   -0.032590
minimum nights                   -0.035211
days_since_last_review           -0.650830
Name: review rate number, dtype: float64


In [48]:
from sklearn.linear_model import LinearRegression

df_non_null = df[df['review rate number'].notnull()]

if not df_non_null.empty:
    X = df_non_null[['days_since_last_review']]  # safe
    y = df_non_null['review rate number']

    model = LinearRegression()
    model.fit(X, y)

    df.loc[df['review rate number'].isnull(), 'review rate number'] = model.predict(
        df.loc[df['review rate number'].isnull(), ['days_since_last_review']]
    )


In [49]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                     59
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [50]:
df['calculated host listings count']

0         6.0
1         2.0
2         1.0
3         1.0
4         1.0
         ... 
102594    1.0
102595    2.0
102596    1.0
102597    1.0
102598    1.0
Name: calculated host listings count, Length: 102591, dtype: float64

In [51]:
#Pearson Correlation with number of review

# Select only numeric columns (excluding id-like columns if needed)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Drop ID columns if present
drop_cols = ['id', 'host id']
numeric_cols = [c for c in numeric_cols if c not in drop_cols]

# Pearson correlation
pearson_corr = df[numeric_cols].corr()['number of reviews'].sort_values(ascending=False)

print("==== Pearson Correlation with number of reviews ====\n")
print(pearson_corr)

==== Pearson Correlation with number of reviews ====

number of reviews                 1.000000
reviews per month                 0.618302
review rate number                0.154535
availability 365                  0.099317
long                              0.069299
price                             0.005394
service fee                       0.005284
Construction year                 0.002022
lat                              -0.025085
minimum nights                   -0.049984
calculated host listings count   -0.080763
days_since_last_review           -0.315329
Name: number of reviews, dtype: float64


In [52]:
df_non_null = df[df['number of reviews'].notnull()]

if not df_non_null.empty:
    X = df_non_null[['reviews per month']]  
    y = df_non_null['number of reviews']

    model = LinearRegression()
    model.fit(X, y)

    df.loc[df['number of reviews'].isnull(), 'number of reviews'] = model.predict(
        df.loc[df['number of reviews'].isnull(), ['reviews per month']]
    )

In [53]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                       409
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [54]:
df.nunique()

id                                102050
NAME                               61273
host id                           102049
host_identity_verified                 2
host name                          13190
neighbourhood group                    7
neighbourhood                        224
lat                                21991
long                               17774
country                                1
country code                           1
instant_bookable                       2
cancellation_policy                    3
room type                              4
Construction year                     20
price                               1151
service fee                          231
minimum nights                       153
number of reviews                    502
last review                         2476
reviews per month                   1017
review rate number                   108
calculated host listings count        78
availability 365                     438
house_rules     

In [55]:
#Pearson Correlation with minimum nights

# Select only numeric columns (excluding id-like columns if needed)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Drop ID columns if present
drop_cols = ['id', 'host id']
numeric_cols = [c for c in numeric_cols if c not in drop_cols]

# Pearson correlation
pearson_corr = df[numeric_cols].corr()['minimum nights'].sort_values(ascending=False)

print("==== Pearson Correlation with minimum nights ====\n")
print(pearson_corr)

==== Pearson Correlation with minimum nights ====

minimum nights                    1.000000
calculated host listings count    0.084847
availability 365                  0.063544
days_since_last_review            0.024969
lat                               0.014889
Construction year                -0.000489
price                            -0.003472
service fee                      -0.003652
review rate number               -0.035089
long                             -0.039639
number of reviews                -0.049976
reviews per month                -0.092276
Name: minimum nights, dtype: float64


In [56]:
df['minimum nights'].skew()

np.float64(86.69860277272363)

In [57]:
# columns_to_remove = ['id', 'license']  # replace with actual column names
# df = df.drop(columns_to_remove, axis=1)
# columns_to_remove = ['NAME', 'host id', 'host name']  # replace with actual column names
# df = df.drop(columns_to_remove, axis=1)
# columns_to_remove = ['country code', 'country']  # replace with actual column names
# df = df.drop(columns_to_remove, axis=1)
#df = df.drop(columns=['last review'])

In [58]:
df.groupby('room type')['minimum nights'].skew()

room type
Entire home/apt     50.930103
Hotel room           2.598347
Private room       119.754048
Shared room         31.267920
Name: minimum nights, dtype: float64

In [59]:
df['minimum nights'] = df.groupby('room type')['minimum nights'] \
                         .transform(lambda x: x.fillna(x.median()))


In [60]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                247
service fee                          273
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [61]:
df.groupby('room type')['price'].skew()

room type
Entire home/apt    0.001499
Hotel room        -0.010485
Private room       0.003832
Shared room       -0.054454
Name: price, dtype: float64

In [62]:
df.groupby('room type')['service fee'].skew()

room type
Entire home/apt    0.002334
Hotel room         0.023653
Private room       0.004324
Shared room       -0.056965
Name: service fee, dtype: float64

In [63]:
df['price'] = df.groupby('room type')['price'].transform(
    lambda x: x.fillna(x.mean())
)


In [64]:
df['service fee'] = df.groupby('room type')['service fee'].transform(
    lambda x: x.fillna(x.mean())
)

In [65]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [66]:
skew_df = df.groupby(['neighbourhood', 'room type'])['Construction year'].skew().reset_index(name='skewness')

filtered_skew = skew_df[(skew_df['skewness'] > 1) | (skew_df['skewness'] < -1)]
filtered_skew

,neighbourhood,room type,skewness
20,Bay Terrace,Entire home/apt,1.668034
41,Belmont,Shared room,1.732051
45,Bergen Beach,Entire home/apt,1.120025
46,Bergen Beach,Private room,-1.206607
56,Briarwood,Shared room,-2.000000
...,...,...,...
543,West Farms,Entire home/apt,-1.704387
544,West Farms,Private room,-1.732051
551,Westerleigh,Entire home/apt,-1.732051
561,Willowbrook,Entire home/apt,1.732051


In [67]:
skew_df = (
    df.groupby(['neighbourhood', 'room type'])['Construction year']
      .skew()
      .reset_index(name='skewness')
)

In [72]:
high_skew = skew_df[skew_df['skewness'].abs() > 1]
high_skew.head()

,neighbourhood,room type,skewness
20,Bay Terrace,Entire home/apt,1.668034
41,Belmont,Shared room,1.732051
45,Bergen Beach,Entire home/apt,1.120025
46,Bergen Beach,Private room,-1.206607
56,Briarwood,Shared room,-2.000000


In [73]:
# First compute median per group
median_by_group = (
    df.groupby(['neighbourhood', 'room type'])['Construction year']
      .transform('median')
)


In [74]:
high_skew_groups = set(zip(high_skew['neighbourhood'], high_skew['room type']))

In [75]:
df['Construction year'] = df.apply(
    lambda row: median_by_group[row.name]
    if pd.isna(row['Construction year']) 
       and (row['neighbourhood'], row['room type']) in high_skew_groups
    else row['Construction year'],
    axis=1
)

In [76]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [77]:
import pandas as pd

# --- 1. Compute skewness ---
skew_df = (
    df.groupby(['neighbourhood', 'room type'])['Construction year']
      .skew()
      .reset_index(name='skewness')
)

print("\n===== Skewness per group =====")
print(skew_df.sort_values("skewness", ascending=False))

# --- 2. Select high-skew groups ---
high_skew = skew_df[skew_df['skewness'].abs() > 1]

print("\n===== High-skew groups (abs(skew) > 1) =====")
print(high_skew)

high_skew_groups = set(zip(high_skew['neighbourhood'], high_skew['room type']))
print("\nHigh skew groups as tuples:", high_skew_groups)

# --- 3. NaN counts BEFORE fill ---
nan_before = df['Construction year'].isna().sum()
print(f"\nNaN count BEFORE fill: {nan_before}")

# --- 4. Which NaN rows belong to high-skew groups ---
nan_rows = df[df['Construction year'].isna()]
nan_in_high_skew = nan_rows[
    nan_rows[['neighbourhood','room type']]
    .apply(tuple, axis=1)
    .isin(high_skew_groups)
]

print("\n===== NaN rows in high-skew groups =====")
print(nan_in_high_skew)

# --- 5. Calculate median for each group ---
median_by_group = (
    df.groupby(['neighbourhood', 'room type'])['Construction year']
      .transform('median')
)

# --- 6. Apply fill ---
df['Construction year'] = df.apply(
    lambda row: median_by_group[row.name]
    if pd.isna(row['Construction year']) 
       and (row['neighbourhood'], row['room type']) in high_skew_groups
    else row['Construction year'],
    axis=1
)

# --- 7. NaN counts AFTER fill ---
nan_after = df['Construction year'].isna().sum()
print(f"\nNaN count AFTER fill: {nan_after}")

print("\n===== Number of NaNs filled =====")
print(nan_before - nan_after)



===== Skewness per group =====
          neighbourhood        room type  skewness
387           Olinville      Shared room  2.236068
77             Canarsie      Shared room  2.236068
226        Graniteville  Entire home/apt  2.191825
178        Emerson Hill  Entire home/apt  2.173644
350          Morrisania      Shared room  2.131628
..                  ...              ...       ...
520  University Heights      Shared room       NaN
522     Upper East Side       Hotel room       NaN
552         Westerleigh     Private room       NaN
569            Woodlawn      Shared room       NaN
573            Woodside      Shared room       NaN

[574 rows x 3 columns]

===== High-skew groups (abs(skew) > 1) =====
    neighbourhood        room type  skewness
20    Bay Terrace  Entire home/apt  1.668034
41        Belmont      Shared room  1.732051
45   Bergen Beach  Entire home/apt  1.120025
46   Bergen Beach     Private room -1.206607
56      Briarwood      Shared room -2.000000
..            ..

In [78]:
nan_rows = df[df['Construction year'].isna()][['neighbourhood', 'room type']]
nan_groups = set(zip(nan_rows['neighbourhood'], nan_rows['room type']))

print("Groups with NaN rows:", nan_groups)
print("Groups with high skew:", high_skew_groups)
print("Intersection:", nan_groups & high_skew_groups)


Groups with NaN rows: {('Prospect Heights', 'Private room'), ('Windsor Terrace', 'Entire home/apt'), ('Fort Greene', 'Entire home/apt'), ('Clinton Hill', 'Entire home/apt'), ('Crown Heights', 'Private room'), ('Bushwick', 'Entire home/apt'), ('Flatbush', 'Entire home/apt'), ('Gowanus', 'Entire home/apt'), ('Sunnyside', 'Private room'), ('Jamaica', 'Private room'), ('Washington Heights', 'Entire home/apt'), ('University Heights', 'Private room'), ('Woodhaven', 'Private room'), ('East Village', 'Entire home/apt'), ('Lower East Side', 'Private room'), ('Midtown', 'Private room'), ('Williamsburg', 'Private room'), ('East Flatbush', 'Entire home/apt'), ('Upper East Side', 'Entire home/apt'), ('Nolita', 'Private room'), ('Greenpoint', 'Entire home/apt'), ('East New York', 'Private room'), ('Astoria', 'Entire home/apt'), ('West Village', 'Entire home/apt'), ('Sunset Park', 'Entire home/apt'), ('Park Slope', 'Private room'), ('Harlem', 'Entire home/apt'), ('Prospect-Lefferts Gardens', 'Private

In [80]:
# 1. Compute skewness for each 3-column group
skew_df = (
    df.groupby(['neighbourhood group', 'neighbourhood', 'room type'])['Construction year']
      .skew()
      .reset_index(name='skewness')
)

# 2. Filter high-skew groups
high_skew = skew_df[skew_df['skewness'].abs() > 1]

# 3. Create a set of tuples for fast lookup
high_skew_groups = set(
    zip(high_skew['neighbourhood group'],
        high_skew['neighbourhood'],
        high_skew['room type'])
)

print("High skew groups:", high_skew_groups)

# 4. Compute median for all groups
median_by_group = (
    df.groupby(['neighbourhood group','neighbourhood', 'room type'])['Construction year']
      .transform('median')
)

# 5. Log NaN rows and their groups
nan_rows = df[df['Construction year'].isna()]
print("\nNaN rows count:", len(nan_rows))

nan_groups = set(
    zip(nan_rows['neighbourhood group'],
        nan_rows['neighbourhood'],
        nan_rows['room type'])
)
print("Groups with NaN:", nan_groups)

# 6. Intersection check
intersection = nan_groups & high_skew_groups
print("Groups where we SHOULD fill:", intersection)

# 7. Apply filling only to those groups
df['Construction year'] = df.apply(
    lambda row: median_by_group[row.name]
    if pd.isna(row['Construction year']) 
       and (row['neighbourhood group'], row['neighbourhood'], row['room type']) in high_skew_groups
    else row['Construction year'],
    axis=1
)

print("\nTotal NaN after filling:", df['Construction year'].isna().sum())


High skew groups: {('Bronx', 'Melrose', 'Private room'), ('Bronx', 'West Farms', 'Entire home/apt'), ('Staten Island', 'Grymes Hill', 'Entire home/apt'), ('Staten Island', 'Clifton', 'Private room'), ('Brooklyn', 'Bergen Beach', 'Entire home/apt'), ('Bronx', 'Olinville', 'Shared room'), ('Staten Island', 'New Dorp Beach', 'Entire home/apt'), ('Staten Island', 'Oakwood', 'Entire home/apt'), ('Queens', 'Douglaston', 'Private room'), ('Staten Island', 'Tottenville', 'Private room'), ('Bronx', 'Norwood', 'Entire home/apt'), ('Staten Island', 'Todt Hill', 'Entire home/apt'), ('Brooklyn', 'Red Hook', 'Shared room'), ('Staten Island', 'Emerson Hill', 'Entire home/apt'), ('Brooklyn', 'Sea Gate', 'Private room'), ('Manhattan', 'Nolita', 'Shared room'), ('Bronx', 'Soundview', 'Entire home/apt'), ('Bronx', 'Highbridge', 'Entire home/apt'), ('Bronx', 'Van Nest', 'Shared room'), ('Staten Island', 'Graniteville', 'Entire home/apt'), ('Bronx', 'West Farms', 'Private room'), ('Bronx', 'Kingsbridge', '

In [81]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                    213
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [82]:
df['Construction year'] = df.groupby(
    ['neighbourhood group', 'neighbourhood', 'room type']
)['Construction year'].transform(
    lambda x: x.fillna(x.median())
)

In [83]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                      0
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                     448
house_rules     

In [84]:
df['availability 365']

0         286.0
1         228.0
2         352.0
3         322.0
4         289.0
          ...  
102594    227.0
102595    395.0
102596    342.0
102597    386.0
102598     69.0
Name: availability 365, Length: 102591, dtype: float64

In [85]:
df.groupby('room type')['availability 365'].skew()

room type
Entire home/apt    0.767922
Hotel room        -0.417073
Private room       0.537839
Shared room        0.156690
Name: availability 365, dtype: float64

In [87]:
df.groupby(['neighbourhood group', 'neighbourhood', 'room type'])['availability 365'].skew()

neighbourhood group  neighbourhood  room type      
Bronx                Allerton       Entire home/apt   -0.059782
                                    Private room       0.166717
                     Baychester     Entire home/apt    0.996220
                                    Private room      -0.554146
                     Belmont        Entire home/apt   -1.221696
                                                         ...   
Staten Island        Westerleigh    Private room            NaN
                     Willowbrook    Entire home/apt    1.732051
                     Woodrow        Entire home/apt    1.501539
brookln              South Slope    Private room            NaN
manhatan             Chelsea        Private room            NaN
Name: availability 365, Length: 576, dtype: float64

In [89]:
df['availability 365'] = df.groupby(
    ['neighbourhood group', 'neighbourhood', 'room type']
)['availability 365'].transform(
    lambda x: x.fillna(x.median())
)

In [90]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                      0
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                       0
house_rules     

In [91]:
df['calculated host listings count']

0         6.0
1         2.0
2         1.0
3         1.0
4         1.0
         ... 
102594    1.0
102595    2.0
102596    1.0
102597    1.0
102598    1.0
Name: calculated host listings count, Length: 102591, dtype: float64

In [92]:
df['calculated host listings count'] = df.groupby('host id')[
    'calculated host listings count'
].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [93]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                      0
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count       319
availability 365                       0
house_rules     

In [94]:
df['calculated host listings count'] = df.groupby(
    ['neighbourhood group', 'neighbourhood']
)['calculated host listings count'].transform(
    lambda x: x.fillna(x.median())
)

In [95]:
df.isna().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified                 0
host name                            406
neighbourhood group                    0
neighbourhood                          0
lat                                    0
long                                   0
country                              532
country code                         131
instant_bookable                       0
cancellation_policy                    0
room type                              0
Construction year                      0
price                                  0
service fee                            0
minimum nights                         0
number of reviews                      0
last review                        15892
reviews per month                      0
review rate number                     0
calculated host listings count         0
availability 365                       0
house_rules     

In [96]:
columns_to_remove = ['id', 'license', 'NAME', 'host id', 'host name', 'country code', 'country', 'last review']  # replace with actual column names
df = df.drop(columns_to_remove, axis=1)
# columns_to_remove = ['NAME', 'host id', 'host name']  # replace with actual column names
# df = df.drop(columns_to_remove, axis=1)
# columns_to_remove = ['country code', 'country']  # replace with actual column names
# df = df.drop(columns_to_remove, axis=1)
#df = df.drop(columns=['last review'])

In [97]:
df.isna().sum()

host_identity_verified            0
neighbourhood group               0
neighbourhood                     0
lat                               0
long                              0
instant_bookable                  0
cancellation_policy               0
room type                         0
Construction year                 0
price                             0
service fee                       0
minimum nights                    0
number of reviews                 0
reviews per month                 0
review rate number                0
calculated host listings count    0
availability 365                  0
house_rules                       0
days_since_last_review            0
dtype: int64

In [98]:
df.dtypes

host_identity_verified             object
neighbourhood group                object
neighbourhood                      object
lat                               float64
long                              float64
instant_bookable                     bool
cancellation_policy                object
room type                          object
Construction year                 float64
price                             float64
service fee                       float64
minimum nights                    float64
number of reviews                 float64
reviews per month                 float64
review rate number                float64
calculated host listings count    float64
availability 365                  float64
house_rules                        object
days_since_last_review            float64
dtype: object